## Statistiche descrittive

Consideriamo nuovamente il DataFrame degli [incidenti](http://dati.comune.milano.it/dataset/9f7bcc9c-20a4-4e48-a7cd-99b15ed11102/resource/38d2171d-1067-4252-9f96-02867a2cc617/download/ds177_trafficotrasporti_incidenti_stradali_persone_infortunate_mese_zona_2001-2016.json).

In [12]:
import pandas as pd
incidenti = pd.read_json("data/incidenti.json")

La funzione `describe` permette di iniziare l'esplorazione del dataset, calcolando alcune statistiche essenziali.

In [13]:
incidenti.describe()

,Anno,Feriti,Incidenti,Mese,Morti,Zona
count,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1728.000000
mean,2008.500000,145.287500,108.352083,6.500000,0.515104,5.000000
std,4.610973,65.982854,48.726844,3.452952,0.752125,2.582736
min,2001.000000,22.000000,17.000000,1.000000,0.000000,1.000000
25%,2004.750000,96.000000,73.000000,3.750000,0.000000,3.000000
50%,2008.500000,137.000000,103.000000,6.500000,0.000000,5.000000
75%,2012.250000,188.000000,140.000000,9.250000,1.000000,7.000000
max,2016.000000,557.000000,439.000000,12.000000,4.000000,9.000000


Notare che `count` conta il numero di righe per cui la variabile non è nulla.

## Rimuovere valori mancanti

La presenza di valori mancanti può creare problemi. Per questo motivo possiamo usare `dropna` per rimuovere tutte le righe che contengono valori mancanti.

In [14]:
incidenti_puliti = incidenti.dropna()
incidenti_puliti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona
1,2001,209,171,1,0,1.0
2,2001,115,91,1,1,2.0
3,2001,187,141,1,1,3.0


Stiamo però attenti che il nuovo DataFrame potrebbe avere caratteristiche diverse dal DataFrame originale. Andiamo a calcolare il numero medio di `Feriti` e di `Incidenti` per i DataFrame `incidenti` e `incidenti_puliti`.

In [15]:
incidenti.describe()[['Feriti', 'Incidenti']].loc['mean']

Feriti       145.287500
Incidenti    108.352083
Name: mean, dtype: float64

In [16]:
incidenti_puliti.describe()[['Feriti', 'Incidenti']].loc['mean']

Feriti       152.524306
Incidenti    113.753472
Name: mean, dtype: float64

## Creare una nuova colonna

Per creare una nuova colonna a partire da altre già esistenti, bisogna scrivere l'espressione corrispondente. Ad esempio, per creare una colonna con il numero di feriti per incidente, bisogna scrivere

In [17]:
incidenti['rapporto'] = incidenti['Feriti'] / incidenti['Incidenti']

Come usuale, è opportuno verificare il risultato dell'operazione, guardando le prime righe del dataset.

In [18]:
incidenti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto
0,2001,69,50,1,1,NaN,1.380000
1,2001,209,171,1,0,1.0,1.222222
2,2001,115,91,1,1,2.0,1.263736


## Assegnare valori ad una colonna

Un caso particolare si ha quando si vuole assegnare lo stesso valore a tutte le righe di un DataFrame. In questo caso assegnamo il valore `0` ad una nuova colonna che chiamiamo `prova`

In [19]:
incidenti['prova'] = 0

In [20]:
incidenti.head(5)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova
0,2001,69,50,1,1,NaN,1.380000,0
1,2001,209,171,1,0,1.0,1.222222,0
2,2001,115,91,1,1,2.0,1.263736,0
3,2001,187,141,1,1,3.0,1.326241,0
4,2001,154,105,1,0,4.0,1.466667,0


## Convertire una colonna

Possiamo notare che la colonna `Zona` contiene numeri con virgola (*float*). Siccome in realtà `Zona` è un numero intero, convertiamo il numero usando il metodo `astype`. Dobbiamo dare un argomento a `astype` che è un dizionario che associa ad ogni colonna da convertire, il tipo da usare nella conversione.

In [21]:
incidenti_puliti = incidenti_puliti.astype({'Zona': int})
incidenti_puliti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona
1,2001,209,171,1,0,1
2,2001,115,91,1,1,2
3,2001,187,141,1,1,3


Però, `astype` richiede che i valori da convertire siano tutti non nulli: per questo motivo abbiamo lavorato su `incidenti_puliti`, invece che su `incidenti`. 

Cosa succede se si eseguono le stesse operazioni su `incidenti`?

## Operare solo su alcune righe

In [22]:
incidenti = incidenti.astype({'Zona': int})

ValueError: Cannot convert non-finite values (NA or inf) to integer

La presenza di valori mancanti rende impossibile l'esecuzione di `astype`. Possiamo sfruttare il fatto che `incidenti` e `incidenti_puliti` hanno lo stesso indice.

In [23]:
incidenti.loc[:, 'Zonaint'] = incidenti_puliti.loc[:, 'Zona']
incidenti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zonaint
0,2001,69,50,1,1,NaN,1.380000,0,NaN
1,2001,209,171,1,0,1.0,1.222222,0,1.0
2,2001,115,91,1,1,2.0,1.263736,0,2.0


La nuova colonna aggiunta (il risultato di `incidenti_puliti.loc[:, 'Zona'].astype(int)`) contiene solo alcune delle righe di `incidenti`. Ma pandas sfrutta gli indici di entrambi i DataFrame per aggiungere la colonna, mettendo valori mancanti dove non trova un indice corrispondente. Ad esempio, `indici_puliti` non ha una riga con indice `0`.
Notare che `Zonaint` non ha numeri interi. Ciò è dovuto alla presenza di valori nulli (`NaN`). Per avere interi dobbiamo eliminarli.

## Operare solo su alcune righe.

Se non si vuole passare per un DataFrame intermedio come `incidenti_puliti`, diventa necessario estendere la chiamata a `loc` esplicitando anche una selezione delle righe, per invocare `astype(int)` solo sulle righe in cui la zona non sia mancante.
Questo passo permette anche di gestire adeguatamente la zona mancante, che verrà posta a *0*.

In [24]:
incidenti['Zona_int'] = 0
incidenti.loc[incidenti['Zona'].notnull(), 'Zona_int'] = \
incidenti.loc[incidenti['Zona'].notnull(), 'Zona'].astype(int)
incidenti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zonaint,Zona_int
0,2001,69,50,1,1,NaN,1.380000,0,NaN,0
1,2001,209,171,1,0,1.0,1.222222,0,1.0,1
2,2001,115,91,1,1,2.0,1.263736,0,2.0,2


## Creare una categoria

Una colonna viene chiamata *categoria* se i suoi valori permettono di classificare le righe.
Adesso andremo a creare una nuova variabile `tipo` che ha tre valori:

*  `a` se il rapporto fra il numero di feriti e il numero di incidenti è minore di 1.3;
*  `b` se il rapporto è fra 1.3 (incluso) e 1.4 (escluso);
*  `c` se il rapporto è maggiore o uguale a 1.4;

Il modo più semplice, ma più lungo, è simile a quanto fatto in precedenza: per ogni valore possibile di `tipo`, si selezionano le righe e si cambia il valore solo di tali righe.

In [25]:
incidenti.loc[incidenti['rapporto'] < 1.3, 'tipo'] = 'a'
incidenti.loc[(incidenti['rapporto'] >= 1.3) & (incidenti['rapporto'] < 1.4), 'tipo'] = 'b'
incidenti.loc[incidenti['rapporto'] >= 1.4, 'tipo'] = 'c'
incidenti.head()

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zonaint,Zona_int,tipo
0,2001,69,50,1,1,NaN,1.380000,0,NaN,0,b
1,2001,209,171,1,0,1.0,1.222222,0,1.0,1,a
2,2001,115,91,1,1,2.0,1.263736,0,2.0,2,a
3,2001,187,141,1,1,3.0,1.326241,0,3.0,3,b
4,2001,154,105,1,0,4.0,1.466667,0,4.0,4,c


## Cancellare una colonna

In [26]:
del incidenti['Zonaint']

In [27]:
incidenti.head(3)

,Anno,Feriti,Incidenti,Mese,Morti,Zona,rapporto,prova,Zona_int,tipo
0,2001,69,50,1,1,NaN,1.380000,0,0,b
1,2001,209,171,1,0,1.0,1.222222,0,1,a
2,2001,115,91,1,1,2.0,1.263736,0,2,a
